In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
from fractions import Fraction
import time
import matplotlib.pyplot as plt
import itertools as it
from IPython.core.display import HTML


sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST
from Modules import WaveformTools as WFT

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

In [2]:
nh = 4
refRatio = 2
c = 1.
CFL = 0.5
nt = 1
RK = 4
deriv = 'CD'
order = 0
mu = 0.5
sigma = .1
wavenumber = 35
plot = 0
Hans = True

In [3]:
omegaF = BT.Grid(nh * refRatio)
print(Fraction(16, 64).numerator)

1


In [4]:
def Residuals(omegaF, cellNums = [], refRatio = 2, plot = 0):
    
    # Obtain attributes from master refined grid.
    nh_max = omegaF.nh_max
    xFine = omegaF.xCell
    
    # Create other necessary objects from master refined grid information.
    wavesF = WT.MakeWaves(omegaF)
    strings = PT.FixStrings(omegaF, np.eye(nh_max), False)
    
    ks = np.linspace(0.5, nh_max / 2, num = nh_max)
    nh_min = int(nh_max / refRatio)

    # Get savePath.
    numPoints, font, X, savePath = PT.UsefulPlotVals()
    saveName = savePath + 'FirstModes' + str(nh_min) + '.png'
    
    # Determine iterations over grid proportions and create vector for storing maximum residual norms.
    if (cellNums == []):
        iters = nh_min
    else:
        iters = 1
    print ('iters =', iters)
    residNormMax = np.zeros(iters, float)
    iterAxis = (np.arange(iters) + 1) / nh_min
    print(iterAxis)
    
    # Iterate over grid refinement proportions.
    for i in range(iters):
        print('i = ', i)
        
        # Instantiate grid with appropriate grid proportions.
        omegaAMR = BT.Grid(nh_min)
        omegaFAMR = BT.Grid(nh_max)
        if (cellNums == []):
            finehalf = list(np.arange(i + 1))
            finehalfF = list(np.arange(refRatio * (i + 1)))
        else:
            # Modify this for FAMR!
            finehalf = cellNums
        omegaAMR.AddPatch(refRatio, finehalf)
        omegaFAMR.AddPatch(refRatio, finehalfF)
        
        degFreed = omegaAMR.degFreed
        xCell = omegaAMR.xCell
        hs = omegaAMR.h
        cells = omegaAMR.cells[-1]
#         print(cells)
#         print(np.all(hs[0] == hs))
        
        wavesAMR = WT.MakeWaves(omegaAMR)
        wavesFAMR = WT.MakeWaves(omegaFAMR)


        nullspace = OT.FindNullspace(omegaAMR, wavesAMR, Hans = Hans)
        nullspaceF = OT.FindNullspace(omegaFAMR, wavesFAMR, Hans = Hans)
        
        restrictOp = GTT.CoarsenOp(omegaAMR)

        restrictOpF = GTT.CoarsenOp(omegaFAMR)

        exactF = wavesF
        FCoefsExact = FFTT.FourierCoefs(omegaF, wavesF, exactF)
        exactAMR = restrictOp @ wavesF @ FCoefsExact

        FCoefsTest = FFTT.FourierCoefs(omegaAMR, wavesAMR @ nullspace, exactAMR, False)

        testAMR = wavesAMR @ nullspace @ FCoefsTest

        resid = abs(exactAMR - testAMR)
        residNorm = TT.VectorNorm(resid, normType = 'max')

        for j in range(nh_max):
#             print('j = ', j)
            

            if (plot > 2):# and (residNorm[j] > 1e-11)):
                if (j % 1 == 0):
                    fig1, ax1 = plt.subplots(figsize = [12, 4])
                    ax1.plot(xFine, exactF[:, j], lw = 1, color = PT.ColorDefault(0), label = 'Exact Fine Solution')
                    ax1.plot(xCell, exactAMR[:, j], lw = 1, color = PT.ColorDefault(1), linestyle = '--', label = 'Exact AMR Solution')
                    ax1.plot(xCell, testAMR[:, j], lw = 1, color = PT.ColorDefault(2), linestyle = ':', label = 'NUDFT AMR Solution')
                    plt.title('AMR and Fine Representations of ' + strings[j])
                    plt.xlabel('$x$')
                    plt.ylabel('Waveform')
                    plt.legend()

                    fig2, ax2 = plt.subplots(figsize = [12, 4])
                    ax2.plot(xCell, resid[:, j], lw = 1)
                    plt.title('Residuals for ' + strings[j])
                    plt.xlabel('$x$')
                    plt.ylabel('Residuals')
                    plt.show()
                    print('\n\n\n\n')

#         PT.PlotWaves(omegaAMR, waves = wavesAMR, nullspace = nullspace)
#                     if (residNorm[j] > 1e-11):
#                         saveName1 = savePath + 'WaveReps' + str(j) + '.png'
#                         saveName2 = savePath + 'Residuals' + str(j) + '.png'
        #                 fig1.savefig(saveName1, bbox_inches = 'tight', dpi = 300, transparent = True)
        #                 fig2.savefig(saveName2, bbox_inches = 'tight', dpi = 300, transparent = True)
#                         print(exactAMR)
#                         print(testAMR)
                    
#             print('j = ', j)
        residNormMax[i] = TT.VectorNorm(residNorm, normType = 'max')
        if (plot > 1):
            numer = str(Fraction(i + 1, nh_min).numerator)
            denom = str(Fraction(i + 1, nh_min).denominator)
            fig, ax = plt.subplots(figsize = [12, 4])
            plt.plot(ks, residNorm)
            ax.grid(True, axis = 'both', zorder = 0)
            plt.yscale('log')
            plt.title(r'Residual Max Norm by Wavenumber for $\frac{' + numer + '}{' + denom + '}$ Grid Refined')
            plt.ylabel('Residual Norm')
            plt.xlabel('Wavenumber')
            plt.show()
        print('end loop i = ', i)
        print('')
        print('')
    if ((plot > 0) and (cellNums == [])):
        fig, ax = plt.subplots(figsize = [12, 4])
        plt.plot(iterAxis, residNormMax, color = PT.ColorDefault(0), lw = 1)
        plt.scatter(iterAxis, residNormMax, color = PT.ColorDefault(0), s = 8)
        # plt.plot(ks, 1e-13 * np.ones(nh_max, float))
        ax.grid(True, axis = 'both', zorder = 0)
        plt.yscale('log')
        plt.title(r'Residual Max Norm by Proportion of Grid Refined')
        plt.ylabel('Residual Norm')
        plt.xlabel('Grid Proportion')
        fig.savefig(saveName, bbox_inches = 'tight', dpi = 300, transparent = True)
        plt.show()
    return

In [5]:
# Residuals(omegaF, plot = plot)